# 선택 분류

## 소프트맥스 함수
소프트맥스 함수는 로짓값 벡터를 확률 분포 벡터로 변환해주는 비선형 함수이다.  

신경망이 어떤 값을 선택했는지는 로짓값 벡터만으로도 알 수 있지만, 우리는 확률 분포로 변환하는 과정을 거친다.  
이에 대한 두가지 이유가 존재한다.
1. 사용자가 확률 분포를 눈으로 확인하기 싶을 경우
2. 손실 함수에 대한 편미분을 구해야 하는 경우

우리는 위의 두 가지 이유로 소프트맥스 함수가 필요하고 식은 다음과 같이 나타낸다.  

$$ y_i = \frac{e^{x_i}}{e^{x_1} + \cdots + e^{x_n}} $$  

하지만 위의 식은 계산 과정에서 오류를 일으킬 수 있기에 다음과 같은 변형식을 이용한다.  

$$ y_i = \frac{e^{x_i-x_k}}{e^{x_1-x_k} + \cdots + e^{x_n-x_k}} $$  


### 소프트맥스 교차 엔트로피
로짓 벡터 $a_1, a_2, \cdots, a_n$과 정답 벡터 $y_1, y_2, \cdots, y_n$이 존재한다고 가정하자.  
이때, 정답 벡터의 확률 분포가 $P$ 이고 로짓 벡터에 소프트맥스 함수를 적용하여 얻은 확률 분포를 $Q$라 하면  

$$ H(P, Q) = -\sum\,p_ilog\,q_i \approx -\sum\,p_ilog(q_i + \epsilon)$$

### 소프트맥스 교차 엔트로피의 편미분
추정 로짓 벡터가 $x_1, x_2, \cdots, x_n$이고, 데이터로 주어진 정답 벡터가 $p_1, p_2, \cdots, p_n$ 이라고 하자.  
이때 소프트맥스 함수를 이용하여 로직 벡터의 확률 분포를 추정한 값이 $q_1, q_2, \cdots, q_n$ 이라 할 때 소프트맥스 교차 엔트로피의 편미분은 다음과 같다.  

$$ \frac{\partial{H}}{\partial{x_i}} = q_i - p_i $$

## 불량 철판 판별 신경망

### 불량 철판 데이터
철판의 표면 상태가 불량인지 아닌지, 불량이라면 어떤 종류의 결함을 갖고 있는지를 판별하여 철판 상태를 7가지로 분류한 데이터

In [ ]:
!wget -O faults.txt https://archive.ics.uci.edu/ml/machine-learning-databases/00198/Faults.NNA

--2021-01-31 07:32:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00198/Faults.NNA
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299482 (292K) [application/x-httpd-php]
Saving to: ‘faults.txt’

faults.txt          100%[===================>] 292.46K  1.10MB/s    in 0.3s    

2021-01-31 07:32:57 (1.10 MB/s) - ‘faults.txt’ saved [299482/299482]



### 파이썬 모듈 불러들이기

In [ ]:
import numpy as np
import time

np.random.seed(1234)

def randomize():
    np.random.seed(time.time())

### 하이퍼파라미터값의 정의

In [ ]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

### 메인 함수 정의

In [ ]:
def steel_exec(epoch_count=10, mb_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수 정의

In [ ]:
def load_steel_dataset():
    with open('faults.txt', 'r') as f:
        rows = []
        lines = f.readlines()
        for line in lines:
            row = [np.float32(value.strip()) for value in line.split('\t')]
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype='float32')

### 파라미터 초기화 함수 정의

In [ ]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

### 학습 및 평가 함수 정의

In [ ]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

### 학습 및 평가  데이터 획득 함수 정의

In [ ]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

### 학습 실행 함수와 평가 실행 함수 정의

In [ ]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

### 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [ ]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

### 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [ ]:
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

### 정확도 계산 함수의 재정의

In [ ]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

### 소프트맥스 관련 함수 정의

In [ ]:
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

## 실행하기

In [ ]:
steel_exec()

Epoch 1: loss=16.341, accuracy=0.290/0.228
Epoch 2: loss=15.851, accuracy=0.312/0.235
Epoch 3: loss=15.360, accuracy=0.333/0.184
Epoch 4: loss=15.464, accuracy=0.328/0.332
Epoch 5: loss=15.866, accuracy=0.311/0.238
Epoch 6: loss=15.524, accuracy=0.326/0.230
Epoch 7: loss=15.776, accuracy=0.315/0.343
Epoch 8: loss=15.791, accuracy=0.314/0.184
Epoch 9: loss=15.375, accuracy=0.332/0.384
Epoch 10: loss=15.346, accuracy=0.334/0.238

Final Test: final accuracy = 0.238


In [ ]:
LEARNING_RATE = 0.0001
steel_exec()

Epoch 1: loss=16.103, accuracy=0.301/0.256
Epoch 2: loss=15.851, accuracy=0.312/0.458
Epoch 3: loss=15.687, accuracy=0.319/0.463
Epoch 4: loss=15.866, accuracy=0.311/0.248
Epoch 5: loss=15.613, accuracy=0.322/0.248
Epoch 6: loss=15.613, accuracy=0.322/0.169
Epoch 7: loss=15.732, accuracy=0.317/0.210
Epoch 8: loss=15.583, accuracy=0.323/0.361
Epoch 9: loss=15.301, accuracy=0.335/0.363
Epoch 10: loss=15.643, accuracy=0.321/0.381

Final Test: final accuracy = 0.381
